In [ ]:
import os
import pandas as pd



import datareader

import problemsolver

import solutions

In [ ]:
FILE_PREFERENCES = "voorkeuren.xlsx"
FILE_GROUPS_TO = "groepen.xlsx"
FILE_NOT_TOGETHER = "niet_samen.xlsx"

In [ ]:
groups_to = pd.read_excel(FILE_GROUPS_TO, index_col=0).to_dict(orient="index")
processor = datareader.VoorkeurenProcessor(FILE_PREFERENCES)
preferences = processor.process(all_to_groups=list(groups_to.keys()))
students_info = processor.get_students_meta_info()
not_together = datareader.read_not_together(FILE_NOT_TOGETHER)

In [ ]:
df_groups_to = pd.DataFrame.from_dict(groups_to, orient="index")
print(df_groups_to.assign(Totaal=lambda df: df.sum("columns")))


df_students = pd.DataFrame.from_dict(students_info, orient="index")
print(df_students[["Jongen/meisje"]].value_counts())
print(df_students["Stamgroep"].value_counts())

In [ ]:
ps = problemsolver.ProblemSolver(
    preferences,
    students_info,
    groups_to,
    not_together,
    max_imbalance_boys_girls_total=6,
    optimize="least_satisfied",
)


ps.run(n_solutions=2, overwrite=True)
ps.run(distance=10, overwrite=True)

In [ ]:
def jsons_to_excel(folder):
    for file in os.listdir(folder):
        if file.endswith(".json"):
            fname = os.path.join(folder, file)
            sa = solutions.SolutionAnalyzer(
                fname, preferences, processor.input, students_info
            )
            sa.to_excel()


jsons_to_excel(ps.get_solution_name())